# Hacker DataQuest Project#

## Introduction ## 
Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

In [2]:
from csv import reader 
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


## Column Definitions

The columns of this data are:

| Title | Definition |
| --- | --- |
|id | The unique identifier from Hacker News for the post |
|title | The title of the post |
|url|The URL that the posts links to, if it the post has a URL |
|num_points |The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes |
|num_comments |The number of comments that were made on the post |
|author |The username of the person who submitted the post |
|created_at | The date and time at which the post was submitted|


## Seperating The Data

The below code is to seperate the header row from the rest of the data set:

In [3]:
headers = hn[0]
print(headers)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


In [4]:
hn = hn[1:]
print(hn[:5])

[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


We're specifically interested in posts whose titles begin with either `Ask HN` or `Show HN`. Users submit Ask HN posts to ask the Hacker News community a specific question. Below are a couple examples:

`Ask HN: How to improve my personal website?
Ask HN: Am I the only one outraged by Twitter shutting down share counts?
Ask HN: Aby recent changes to CSS that broke mobile?`

Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting. Below are a couple of examples:

`Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'
Show HN: Something pointless I made
Show HN: Shanhu.io, a programming playground powered by e8vm`

We'll compare these two types of posts to determine the following:

Do Ask HN or Show HN receive more comments on average?
Do posts created at a certain time receive more comments on average?

In [5]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    lower_title = title.lower() # Converts title to all lower case
    
    if lower_title.startswith('ask hn'):
        ask_posts.append(row)
    
    elif lower_title.startswith('show hn'):
        show_posts.append(row)
    
    else:
        other_posts.append(row)

print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


## Data Analysis ##

We found above that there are 1744 posts that start with some variation of `Ask HN` and 1162 posts that start with `Show HN`.

Now these are seperated into different lists the total number of comments for each list can be found:

In [6]:
total_ask_comments = 0

for row in ask_posts:
    num = int(row[4])
    total_ask_comments += num
    
avg_ask_comments = total_ask_comments/(len(ask_posts))

print('The average comments on "Ask HN" posts is: ')
print('%.2f' %avg_ask_comments) #To 2.d.p

    

The average comments on "Ask HN" posts is: 
14.04


Similaly for `Show HN` posts:

In [7]:
total_show_comments = 0

for row in show_posts:
    num = int(row[4])
    total_show_comments += num
    
avg_show_comments = total_show_comments/(len(show_posts))

print('The average comments on "Show HN" posts is: ')
print('%.2f' %avg_show_comments) #To 2.d.p

The average comments on "Show HN" posts is: 
10.32


On avergae then, there are roughly 40% more comments on a `Ask HN` post than a `Show HN` post

## Date and Time Analysis

Since Ask posts recieve more comments, the remainder of this analysis will focus on the `Ask_posts` list

The nex step is to determine if ask posts created at a certain time are more likely to attract comments. 

We'll use the following steps to perform this analysis:

- Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
- Calculate the average number of comments ask posts receive by hour created.

In this screen, we'll tackle the first step — calculating the amount of ask posts and comments by hour created. We'll use the `datetime` module to work with the data in the `created_at` column.

In [8]:
import datetime as dt

result_list1 = []
result_list2 = []
result_list = []

for row in ask_posts:
    created_at = row[6]
    num = int(row[4])
    result_list.append([created_at,num])
    

print(result_list[:5])

[['8/16/2016 9:55', 6], ['11/22/2015 13:43', 29], ['5/2/2016 10:14', 1], ['8/2/2016 14:20', 3], ['10/15/2015 16:38', 17]]


In [9]:
counts_by_hour = {}
comments_by_hour = {}
date_format = ('%m/%d/%Y %H:%M')

for row in result_list:
    date = row[0]
    comment = row[1]
    date = dt.datetime.strptime(date,date_format)
    hour = date.strftime('%H')
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = comment
    elif hour in counts_by_hour:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += comment
    
print('Counts by hour:')
print(counts_by_hour)
print('\n')
print('Comments by hour:')
print(comments_by_hour)


Counts by hour:
{'19': 110, '17': 100, '13': 85, '09': 45, '12': 73, '14': 107, '21': 109, '05': 46, '02': 58, '03': 54, '11': 58, '15': 116, '22': 71, '16': 108, '01': 60, '20': 80, '18': 109, '23': 68, '00': 55, '10': 59, '07': 34, '04': 47, '06': 44, '08': 48}


Comments by hour:
{'19': 1188, '17': 1146, '13': 1253, '09': 251, '12': 687, '14': 1416, '21': 1745, '05': 464, '02': 1381, '03': 421, '11': 641, '15': 4477, '22': 479, '16': 1814, '01': 683, '20': 1722, '18': 1439, '23': 543, '00': 447, '10': 793, '07': 267, '04': 337, '06': 397, '08': 492}


We can see that the highest number of posts occur in the afternoon to early evening i.e. 14:00 -> 19:00

The largest number of comments occurs at 1500 coinciding with the most number of posts

## Average Comments Per Post

Next we can break down the average amount of comments per post at each time interval to find the optimum time to post.

In [10]:
avg_by_hour = []

for hour in comments_by_hour:
    avg = comments_by_hour[hour]/counts_by_hour[hour]
    avg_by_hour.append([hour , avg])

print(avg_by_hour)

[['19', 10.8], ['17', 11.46], ['13', 14.741176470588234], ['09', 5.5777777777777775], ['12', 9.41095890410959], ['14', 13.233644859813085], ['21', 16.009174311926607], ['05', 10.08695652173913], ['02', 23.810344827586206], ['03', 7.796296296296297], ['11', 11.051724137931034], ['15', 38.5948275862069], ['22', 6.746478873239437], ['16', 16.796296296296298], ['01', 11.383333333333333], ['20', 21.525], ['18', 13.20183486238532], ['23', 7.985294117647059], ['00', 8.127272727272727], ['10', 13.440677966101696], ['07', 7.852941176470588], ['04', 7.170212765957447], ['06', 9.022727272727273], ['08', 10.25]]


Whilst all the required information is above it is difficult to read as it is not in any discernable order. 

The code below switches the order so that instead of hour:average the list displays average:hour and sorts in descending order

In [15]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1] , row[0]])
print(swap_avg_by_hour)
print('\n')
sorted_swap = sorted(swap_avg_by_hour,reverse=True)

print('Top 5 Hours for Ask Posts Comments')

for avg , hour in sorted_swap[:5]:
    print('{}:00: {:.2f} average comments per post'.format(hour, avg))
    
        


[[10.8, '19'], [11.46, '17'], [14.741176470588234, '13'], [5.5777777777777775, '09'], [9.41095890410959, '12'], [13.233644859813085, '14'], [16.009174311926607, '21'], [10.08695652173913, '05'], [23.810344827586206, '02'], [7.796296296296297, '03'], [11.051724137931034, '11'], [38.5948275862069, '15'], [6.746478873239437, '22'], [16.796296296296298, '16'], [11.383333333333333, '01'], [21.525, '20'], [13.20183486238532, '18'], [7.985294117647059, '23'], [8.127272727272727, '00'], [13.440677966101696, '10'], [7.852941176470588, '07'], [7.170212765957447, '04'], [9.022727272727273, '06'], [10.25, '08']]


Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


## Conclusions

The time at which there is the most amount of posts is also the time at which there is the most user engagement i.e. the largest amount of comments per post on average